## Vanna text to sql

In [1]:
%pip install 'vanna[chromadb,ollama,postgres]'

Note: you may need to restart the kernel to use updated packages.


In [2]:
from vanna.ollama import Ollama
from vanna.chromadb import ChromaDB_VectorStore

/Users/adityakumarraj/Desktop/text2sql/venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [3]:
class MyVanna(ChromaDB_VectorStore, Ollama):
    def __init__(self, config=None):
        ChromaDB_VectorStore.__init__(self, config=config)
        Ollama.__init__(self, config=config)

vn = MyVanna(config={'model': 'mistral', 'allow_llm_to_see_data': True})

In [4]:
a = vn.get_training_data()
print(a)

Empty DataFrame
Columns: [id, question, content, training_data_type]
Index: []


In [5]:
%pip install psycopg2-binary

Note: you may need to restart the kernel to use updated packages.


In [6]:
vn.connect_to_postgres(
    dbname="data",
    user="adityakumarraj",
    password="Aa%%407909013706",
    host="localhost",
    port="5432"
)

In [7]:
# Query to get detailed column information
table_info_sql = """
SELECT 
    column_name,
    data_type
FROM 
    INFORMATION_SCHEMA.COLUMNS 
WHERE 
    table_name = 'nodedata' 
    AND table_schema = 'public'
ORDER BY 
    ordinal_position;
"""

# Execute the query
table_description = vn.run_sql(table_info_sql)
print("Table Structure for public.nodedata:")
print(table_description)

Table Structure for public.nodedata:
    column_name                    data_type
0        nodeid                         text
1      nodename                         text
2   clustername                         text
3  instancetype                         text
4          tags                         text
5       created  timestamp without time zone
6  snapshottime  timestamp without time zone
7      platform                         text


In [8]:
df_information_schema = vn.run_sql("SELECT * FROM INFORMATION_SCHEMA.COLUMNS where table_name='nodedata' AND table_schema='public';")
df_information_schema
# This will break up the information schema into bite-sized chunks that can be referenced by the LLM
plan = vn.get_training_plan_generic(df_information_schema)
plan
# If you like the plan, then uncomment this and run it to train
vn.train(plan=plan)

In [9]:
training_data = vn.get_training_data()
training_data

,id,question,content,training_data_type
0,54f451ac-e1fd-573e-9efe-b95ac41a1d06-doc,None,The following columns are in the nodedata tabl...,documentation


In [10]:
# from vanna.flask import VannaFlaskApp
# app = VannaFlaskApp(vn)
# app.run()

# Agentic LLM

In [11]:
vn.train(documentation='''
The table name is 'nodedata' and it is present in the 'public' schema of the database 'data'.
Note the schema of the table nodedatafor which you will be generating queries:
    nodeid        TEXT,    -- Unique identifier for the node (PRIMARY KEY)
    nodename      TEXT,    -- Name of the node
    clustername   TEXT,    -- Name of the Kubernetes or compute cluster
    instancetype  TEXT,    -- EC2 instance type or VM size (e.g., c5.18xlarge, p4d.24xlarge)
    tags          TEXT,    -- JSON-style string of metadata tags (key/value pairs)
    created       TIMESTAMP WITHOUT TIME ZONE, -- When the node was first created
    snapshottime  TIMESTAMP WITHOUT TIME ZONE, -- When this snapshot of node metadata was taken
    platform      TEXT     -- Runtime platform or orchestration tool (e.g., runai, kubernetes)
)
''')


Adding documentation....


'eb32a74c-4bf1-5e0e-925f-6d738239b1d6-doc'

In [12]:
vn.train(documentation='''
You are an AI assistant trained to generate SQL queries from user queries on table nodedata. The dataset contains instance metadata—including creation and snapshot timestamps—for nodes across clusters. Your goal is to accurately translate user questions into optimized SQL queries while handling potential issues such as:
Primary Key:
• Ensure nodeid is treated as the primary key—each node is uniquely identified.

Date Ranges:
• Interpret relative time windows on the created and snapshottime columns:
  - “Last 7 days” → WHERE snapshottime ≥ now() - interval '7 days'
  - “Created in Q1” → WHERE created BETWEEN '2025-01-01' AND '2025-03-31'
  - “Between date X and Y” → inclusive BETWEEN on created or snapshottime.

HAVING Clause Rule:
• Do NOT use column aliases (e.g., node_count) in the HAVING clause. Instead, repeat the full aggregation function (e.g., HAVING COUNT(nodeid) > 5).


In queries always refer to the table as 'nodedata' not as 'data.nodedata'.
Generate syntactically correct PostgreSQL queries that can be directly executed.
Always verify column existence and ensure that queries are efficient and correct.
If a query involves calculations, wrap division operations with NULLIF(denominator, 0) to avoid errors.
''')

Adding documentation....


'b7dca632-4126-522e-9346-a62d6168a75b-doc'

In [13]:
training_data = vn.get_training_data()
print(training_data)

                                         id question  \
0  54f451ac-e1fd-573e-9efe-b95ac41a1d06-doc     None   
1  eb32a74c-4bf1-5e0e-925f-6d738239b1d6-doc     None   
2  b7dca632-4126-522e-9346-a62d6168a75b-doc     None   

                                             content training_data_type  
0  The following columns are in the nodedata tabl...      documentation  
1  \nThe table name is 'nodedata' and it is prese...      documentation  
2  \nYou are an AI assistant trained to generate ...      documentation  


In [14]:
# Test questions to verify different aspects of the model
test_questions = [
    # Basic counting and grouping
    "How many nodes are there per cluster?",
    
    # Time-based analysis
    "Show me the nodes created in the last 7 days",
    
    # Multiple conditions
    "What are the instance types used in each platform?",
    
    # Aggregations
    "What is the distribution of instance types?",
    
    # Complex combinations
    "Show me the clusters that have more than 5 nodes, grouped by platform",
    
    # Specific lookups
    "What are all the nodes in the 'sensei-eks01-prod-cluster' cluster?",
    
    # Pattern matching
    "Find all nodes that have 'compute' in their name",
    
    # Time series
    "Show the number of nodes created per day"
]

# Function to test and display results
def test_vanna_model(vn, questions):
    print("Testing Vanna Model with Various Questions\n")
    print("-" * 80)
    
    for i, question in enumerate(questions, 1):
        print(f"\nTest {i}: {question}")
        print("\nGenerated SQL:")
        try:
            sql = vn.generate_sql(question)
            print(sql)
            
            print("\nQuery Result:")
            result = vn.run_sql(sql)
            print(result.head() if not result.empty else "No results found")
            
        except Exception as e:
            print(f"Error: {str(e)}")
        
        print("-" * 80)

# Run the tests
test_vanna_model(vn, test_questions)

# Additional specific test for data validation
print("\nData Validation Tests:")
print("-" * 80)

# Test 1: Check unique platforms
sql_platforms = "SELECT DISTINCT platform, COUNT(*) as node_count FROM public.nodedata GROUP BY platform ORDER BY node_count DESC;"
print("\nPlatform Distribution:")
print(vn.run_sql(sql_platforms))

# Test 2: Check date range
sql_dates = """
SELECT 
    MIN(created) as earliest_date,
    MAX(created) as latest_date,
    COUNT(*) as total_nodes
FROM public.nodedata;
"""
print("\nDate Range and Total Nodes:")
print(vn.run_sql(sql_dates))

# Test 3: Instance type summary
sql_instances = """
SELECT 
    instancetype,
    COUNT(*) as count,
    COUNT(*) * 100.0 / SUM(COUNT(*)) OVER () as percentage
FROM public.nodedata
GROUP BY instancetype
ORDER BY count DESC;
"""
print("\nInstance Type Distribution:")
print(vn.run_sql(sql_instances))

Testing Vanna Model with Various Questions

--------------------------------------------------------------------------------

Test 1: How many nodes are there per cluster?

Generated SQL:


Number of requested results 10 is greater than number of elements in index 3, updating n_results = 3


SQL Prompt: [{'role': 'system', 'content': "You are a PostgreSQL expert. Please help to generate a SQL query to answer the question. Your response should ONLY be based on the given context and follow the response guidelines and format instructions. \n===Additional Context \n\n\nThe table name is 'nodedata' and it is present in the 'public' schema of the database 'data'.\nNote the schema of the table nodedatafor which you will be generating queries:\n    nodeid        TEXT,    -- Unique identifier for the node (PRIMARY KEY)\n    nodename      TEXT,    -- Name of the node\n    clustername   TEXT,    -- Name of the Kubernetes or compute cluster\n    instancetype  TEXT,    -- EC2 instance type or VM size (e.g., c5.18xlarge, p4d.24xlarge)\n    tags          TEXT,    -- JSON-style string of metadata tags (key/value pairs)\n    created       TIMESTAMP WITHOUT TIME ZONE, -- When the node was first created\n    snapshottime  TIMESTAMP WITHOUT TIME ZONE, -- When this snapshot of node metadata wa

Number of requested results 10 is greater than number of elements in index 3, updating n_results = 3


SQL Prompt: [{'role': 'system', 'content': "You are a PostgreSQL expert. Please help to generate a SQL query to answer the question. Your response should ONLY be based on the given context and follow the response guidelines and format instructions. \n===Additional Context \n\n\nYou are an AI assistant trained to generate SQL queries from user queries on table nodedata. The dataset contains instance metadata—including creation and snapshot timestamps—for nodes across clusters. Your goal is to accurately translate user questions into optimized SQL queries while handling potential issues such as:\nPrimary Key:\n• Ensure nodeid is treated as the primary key—each node is uniquely identified.\n\nDate Ranges:\n• Interpret relative time windows on the created and snapshottime columns:\n  - “Last 7 days” → WHERE snapshottime ≥ now() - interval '7 days'\n  - “Created in Q1” → WHERE created BETWEEN '2025-01-01' AND '2025-03-31'\n  - “Between date X and Y” → inclusive BETWEEN on created or snapsho

...

SQL Prompt: [{'role': 'system', 'content': "You are a PostgreSQL expert. Please help to generate a SQL query to answer the question. Your response should ONLY be based on the given context and follow the response guidelines and format instructions. \n===Additional Context \n\n\nThe table name is 'nodedata' and it is present in the 'public' schema of the database 'data'.\nNote the schema of the table nodedatafor which you will be generating queries:\n    nodeid        TEXT,    -- Unique identifier for the node (PRIMARY KEY)\n    nodename      TEXT,    -- Name of the node\n    clustername   TEXT,    -- Name of the Kubernetes or compute cluster\n    instancetype  TEXT,    -- EC2 instance type or VM size (e.g., c5.18xlarge, p4d.24xlarge)\n    tags          TEXT,    -- JSON-style string of metadata tags (key/value pairs)\n    created       TIMESTAMP WITHOUT TIME ZONE, -- When the node was first created\n    snapshottime  TIMESTAMP WITHOUT TIME ZONE, -- When this snapshot of node metadata wa

Number of requested results 10 is greater than number of elements in index 3, updating n_results = 3


SQL Prompt: [{'role': 'system', 'content': "You are a PostgreSQL expert. Please help to generate a SQL query to answer the question. Your response should ONLY be based on the given context and follow the response guidelines and format instructions. \n===Additional Context \n\nThe following columns are in the nodedata table in the data database:\n\n|    | table_catalog   | table_schema   | table_name   | column_name   | data_type                   |\n|---:|:----------------|:---------------|:-------------|:--------------|:----------------------------|\n|  0 | data            | public         | nodedata     | nodeid        | text                        |\n|  1 | data            | public         | nodedata     | nodename      | text                        |\n|  2 | data            | public         | nodedata     | clustername   | text                        |\n|  3 | data            | public         | nodedata     | instancetype  | text                        |\n|  4 | data            | p

Number of requested results 10 is greater than number of elements in index 3, updating n_results = 3


Info: Ollama Response:
model='mistral:latest' created_at='2025-05-19T09:51:09.458676Z' done=True done_reason='stop' total_duration=2352128417 load_duration=12569667 prompt_eval_count=1056 prompt_eval_duration=1453036417 eval_count=54 eval_duration=881883708 message=Message(role='assistant', content=" SELECT nodeid, nodename\n   FROM nodedata\n   WHERE clustername IS NOT NULL\n     AND instancetype IS NOT NULL\n     AND clustername LIKE '%compute%'\n     AND nodename LIKE '%compute%';", images=None, tool_calls=None)
LLM Response:  SELECT nodeid, nodename
   FROM nodedata
   WHERE clustername IS NOT NULL
     AND instancetype IS NOT NULL
     AND clustername LIKE '%compute%'
     AND nodename LIKE '%compute%';
Info: Output from LLM:  SELECT nodeid, nodename
   FROM nodedata
   WHERE clustername IS NOT NULL
     AND instancetype IS NOT NULL
     AND clustername LIKE '%compute%'
     AND nodename LIKE '%compute%'; 
Extracted SQL: SELECT nodeid, nodename
   FROM nodedata
   WHERE clusternam